In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 36.7MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [72]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",
        "Alibaba-NLP/gme-Qwen2-VL-2B-Instruct",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",
        "Alibaba-NLP/gme-Qwen2-VL-2B-Instruct",

# "bert",

]


In [73]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["chn"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [74]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [75]:
all_csv

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...
2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [76]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [77]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [78]:
train_data=all_csv


In [79]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [80]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...
2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [81]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...
2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [82]:
train_data.dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [83]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...,...
2637,2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [84]:
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...
2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [85]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [86]:
train_data['lang'].value_counts()

,count
lang,
chn,2642


In [87]:
sum(train_data['lang'].isna())

0

In [88]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [89]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [90]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [91]:
nan_rows["lang"].value_counts()

,count
lang,


In [92]:
nan_rows.dtypes

,0
id,object
anger,int64
disgust,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [93]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

             id  disgust   fear    joy  sadness  surprise  lang  clean_message
lang                                                                          
chn  0     True    False  False  False    False     False  True           True
     1     True    False  False   True    False     False  True           True
     2     True    False  False  False    False     False  True           True
     3     True    False  False   True    False     False  True           True
     4     True    False  False  False    False     False  True           True
...         ...      ...    ...    ...      ...       ...   ...            ...
     2637  True     True  False  False    False     False  True           True
     2638  True     True  False  False     True     False  True           True
     2639  True    False  False  False     True     False  True           True
     2640  True    False  False  False    False     False  True           True
     2641  True    False  False   True    False     

<ipython-input-93-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [94]:
maxi=train_data['clean_message'].apply(len).max()
maxi

176

In [95]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
2436,chn_train_track_a_02437,1,0,0,1,0,0,chn,评论里其他ip的人少来这里评论谢谢了folded hands folded hands fo...


In [96]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['评论里其他ip的人少来这里评论谢谢了folded hands folded hands folded hands folded hands folded hands folded hands 我没说不放开我反而希望一夜之间全部放开你们赶紧去看演唱会看球赛出国旅游哈反正我家住自建房不怕下水道气溶胶家里囤了够三个人吃两个月的物资朋友是私立医院的老板嘿嘿嘿'],
      dtype=object)

In [97]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [98]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [99]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [100]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [101]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1178      935  2113
disgust      1785      328  2113
fear         2052       61  2113
joy          1678      435  2113
sadness      1824      289  2113
surprise     1964      149  2113


In [102]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [103]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([ 1.1299,  3.2210, 17.3197,  2.4287,  3.6557,  7.0906], device='cuda:0')

In [104]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [105]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


In [106]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [107]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [108]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [109]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [110]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [111]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [112]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [113]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [114]:
USE_LORA=False

In [118]:
!wget https://huggingface.co/Alibaba-NLP/gme-Qwen2-VL-2B-Instruct/resolve/main/gme_inference.py

--2025-01-13 07:47:36--  https://huggingface.co/Alibaba-NLP/gme-Qwen2-VL-2B-Instruct/resolve/main/gme_inference.py
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13152 (13K) [text/plain]
Saving to: ‘gme_inference.py’

gme_inference.py    100%[===================>]  12.84K  --.-KB/s    in 0s      

2025-01-13 07:47:36 (300 MB/s) - ‘gme_inference.py’ saved [13152/13152]



In [119]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
from gme_inference import GmeQwen2VL

In [124]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
# embedding_model = GmeQwen2VL(x)
embedding_model = GmeQwen2VL(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        quantization_config=quantization_config,
        # torch_dtype=torch.bfloat16 if use_gpu else torch.float32,
        # model_kwargs={
        #     "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
        #     # "attn_implementation": "sdpa",
        #     "quantization_config":quantization_config,
        # },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

Alibaba-NLP/gme-Qwen2-VL-2B-Instruct


`low_cpu_mem_usage` was None, now default to True since model is quantized.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
# a=

In [127]:
device

'cuda'

In [129]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]
        # return self.tokenizer(
        #     self.texts[idx],
        #     truncation=True,
        #     padding="max_length",
        #     max_length=self.max_length,
        #     return_tensors="pt"
        # )


In [136]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    # dataset = TextDataset(texts)
    # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return model.get_text_embeddings(texts=texts,batch_size=batch_size)
    # embeddings = []
    # with torch.no_grad():
    #     for batch in tqdm(dataloader, desc="Extracting embeddings"):
    #         # Move batch to device
    #         # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
    #         # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

    #         # Forward pass through the model
    #         cls_embeddings =
    #         embeddings.append(cls_embeddings)

    # return np.vstack(embeddings)

In [137]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)
x_valid_embeddings = extract_embeddings(x_valid, embedding_model, batch_size=32)

encode:   0%|          | 0/67 [00:00<?, ?it/s]

encode:   0%|          | 0/17 [00:00<?, ?it/s]

In [138]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)
y_valid_flat = np.array(y_valid)

In [139]:
import xgboost as xgb

In [149]:
from sklearn.ensemble import RandomForestClassifier

rf_models = {}
predictions = {}

for i, label in enumerate(label_columns):
    print(f"Training RandomForest for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    # Initialize and train RandomForest classifier
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=6,
        class_weight=class_weight,  # Apply class weight for imbalanced data
        random_state=42,
        n_jobs=-1  # Use all available cores
    )
    rf_model.fit(x_train_embeddings, y_train_flat[:, i])
    rf_models[label] = rf_model

    # Predict on validation data
    predictions[label] = rf_model.predict(x_valid_embeddings)

Training RandomForest for label: anger
Training RandomForest for label: disgust
Training RandomForest for label: fear
Training RandomForest for label: joy
Training RandomForest for label: sadness
Training RandomForest for label: surprise


In [150]:
# Evaluate the models
results = []
for i, label in enumerate(label_columns):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_valid_flat[:, i],
        predictions[label],
        average="binary"
    )
    accuracy = accuracy_score(y_valid_flat[:, i], predictions[label])
    results.append({
        "Label": label,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [151]:
# Display evaluation results
results_df = pd.DataFrame(results)
print(results_df)

      Label  Precision    Recall  F1 Score  Accuracy
0     anger   0.641892  0.390947  0.485934  0.620038
1   disgust   0.000000  0.000000  0.000000  0.858223
2      fear   0.000000  0.000000  0.000000  0.981096
3       joy   0.800000  0.042553  0.080808  0.827977
4   sadness   0.000000  0.000000  0.000000  0.877127
5  surprise   0.000000  0.000000  0.000000  0.945180


In [152]:
print(f"{x} evaluates:")

for k, res in enumerate(results):

  print(f"Class '{res['Label']}': Precision: {res['Precision']:.4f}, Recall: {res['Recall']:.4f}, F1: {res['F1 Score']:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

Alibaba-NLP/gme-Qwen2-VL-2B-Instruct evaluates:
Class 'anger': Precision: 0.6419, Recall: 0.3909, F1: 0.4859
Class 'disgust': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'fear': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'joy': Precision: 0.8000, Recall: 0.0426, F1: 0.0808
Class 'sadness': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'surprise': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Overall accuracy: 0.9452
-------------------------------------------------------------------------



In [157]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()

    if len(lang_valid_data) == 0:
        continue

    # Extract embeddings for the current language's validation data
    lang_x_valid_embeddings = extract_embeddings(lang_x_valid, embedding_model)
    print()
    # Initialize metrics storage
    all_precisions, all_recalls, all_f1_scores, all_accuracies = [], [], [], []

    # Evaluate each label separately
    for i, label in enumerate(label_columns):
        # Predict using the corresponding XGBoost model
        lang_predictions = rf_models[label].predict(lang_x_valid_embeddings)

        # Compute metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.array(lang_y_valid)[:, i],  # True labels for this label
            lang_predictions,
            average="binary",
            zero_division=0
        )
        accuracy = accuracy_score(np.array(lang_y_valid)[:, i], lang_predictions)

        # Store metrics
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)
        all_accuracies.append(accuracy)

    # Aggregate metrics (average across all labels for the language)
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [np.mean(all_precisions)],
        "Recall": [np.mean(all_recalls)],
        "F1_score": [np.mean(all_f1_scores)],
        "Accuracy": [np.mean(all_accuracies)]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)


encode:   0%|          | 0/17 [00:00<?, ?it/s]


  Language  Precision   Recall  F1_score  Accuracy
0      chn   0.240315  0.07225  0.094457  0.851607


<ipython-input-157-4707a06c52e3>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


In [158]:
# Create a zip archive of all the model files
!zip rf_models.zip rf_model_*.json

	zip warning: name not matched: rf_model_*.json

zip error: Nothing to do! (rf_models.zip)


In [161]:
import pickle
# Save the XGBoost models if needed
for label, model in rf_models.items():
    with open(f"rf_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [162]:
!zip rf_models_pkl.zip rf_model_*.pkl

  adding: rf_model_anger.pkl (deflated 66%)
  adding: rf_model_disgust.pkl (deflated 67%)
  adding: rf_model_fear.pkl (deflated 72%)
  adding: rf_model_joy.pkl (deflated 67%)
  adding: rf_model_sadness.pkl (deflated 68%)
  adding: rf_model_surprise.pkl (deflated 69%)
